In [1]:
import os
from glob import glob
import numpy as np
import pandas as pd
from read_dataset import build_df
from utils import CFG
from sklearn.model_selection import train_test_split
import torch
from models import *
from main import *
from tqdm import tqdm
from torch.utils.data import DataLoader
from datapreprocess import *


# Parameters
params = {"batch_size": 64, "shuffle": True, "num_workers": 6}
max_epochs = 100
NUM_CLASSES = 11
RESNET_OUT_FEATURES = 1000



IMG_FILES = glob(CFG.img_path + "/*.jpg")
XML_FILES = glob(CFG.xml_path + "/*.xml")
df, classes = build_df(XML_FILES)
data = df.to_numpy()

    # input and target
# input and target
input = df[["img_path"]].values
input = np.squeeze(input)
# input = input.reset_index()
target = df[["xmin", "ymin", "xmax", "ymax", "label"]].values.astype(np.int32)


    # splitting data
    # train_data, validation_data, test_data = split_dataset(input,target,True)
train_df, test_df = split_dataset(input, target, validation=False)


XML_FILES_Size:  11000
IMG_FILES_Size:  11000


In [2]:
train_df[1][4] # location and label of the fourth item

array([1032, 1673, 1188, 1800,    1], dtype=int32)

In [3]:
# train_dataset  = ResNetDataset(train_df, transform=None)
# test_dataset = ResNetDataset(test_df, transform=None)
# train_dataset[0]

In [5]:

transform = preprocess_image
train_dataset  = ResNetDataset(train_df, transform=transform)
test_dataset = ResNetDataset(test_df, transform=transform)
# train_dataset[0][1]

In [ ]:

# Dataloaders
train_dataloader = DataLoader(
    train_df, params["batch_size"], num_workers=params["num_workers"], shuffle=params["shuffle"],
)
# validation_dataloader = torch.utils.data.DataLoader(validation_data, params['batch_size'],num_workers=params['num_workers'])
test_dataloader = DataLoader(
    test_df, params["batch_size"], num_workers=params["num_workers"]
)


In [ ]:

for x, y in tqdm(train_dataloader):
    x  = torch.Variable(x, requires_grad=True).to(device)
    y  = torch.LongTensor(y).to(device)